# Setup

In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
import csv
import sqlite3

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from ipywidgets import interact

from flask import Flask, jsonify


In [5]:
# create and save database
#execfile("Resources/Setup_and_Organization/Database_Setup.py")

In [ ]:
conn = sqlite3.connect('sleep_health_lifestyle.db')


In [7]:
# create engine to newly created database.sqlite
engine = create_engine("sqlite:///Resources/database.sqlite")
conn=engine.connect()


# Explore Database

In [8]:
# explore database
inspector = inspect(engine)
inspector.get_table_names()

['activity',
 'bmi',
 'gender',
 'health',
 'occupation',
 'person',
 'sleep',
 'sleep_disorder']

In [9]:
inspector.get_columns('activity')

[{'name': 'index',
  'type': BIGINT(),
  'nullable': True,
  'default': None,
  'primary_key': 0},
 {'name': 'id',
  'type': BIGINT(),
  'nullable': True,
  'default': None,
  'primary_key': 0},
 {'name': 'person_id',
  'type': BIGINT(),
  'nullable': True,
  'default': None,
  'primary_key': 0},
 {'name': 'activity_level',
  'type': BIGINT(),
  'nullable': True,
  'default': None,
  'primary_key': 0},
 {'name': 'daily_steps',
  'type': BIGINT(),
  'nullable': True,
  'default': None,
  'primary_key': 0}]

# Reflect and query data

In [10]:
# Create a base class for declarating class definitions to produce Table objects
Base = declarative_base()

# Create Database Connection
#engine = create_engine(f"sqlite:///Resources/database.sqlite")
Base.metadata.create_all(engine) #Carlos recommends after creating class

In [ ]:
engine

In [11]:
#engine = create_engine(f"sqlite:///Resources/database.sqlite")
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(autoload_with=engine)
print("keys", Base.classes.keys())

Base.metadata.create_all(engine) #Carlos recommends after creating class


keys []


In [13]:
Base.metadata.tables

FacadeDict({'activity': Table('activity', MetaData(), Column('index', BIGINT(), table=<activity>), Column('id', BIGINT(), table=<activity>), Column('person_id', BIGINT(), table=<activity>), Column('activity_level', BIGINT(), table=<activity>), Column('daily_steps', BIGINT(), table=<activity>), schema=None), 'bmi': Table('bmi', MetaData(), Column('index', BIGINT(), table=<bmi>), Column('id', BIGINT(), table=<bmi>), Column('bmi_name', TEXT(), table=<bmi>), schema=None), 'gender': Table('gender', MetaData(), Column('index', BIGINT(), table=<gender>), Column('id', BIGINT(), table=<gender>), Column('gender_name', TEXT(), table=<gender>), schema=None), 'health': Table('health', MetaData(), Column('index', BIGINT(), table=<health>), Column('id', BIGINT(), table=<health>), Column('person_id', BIGINT(), table=<health>), Column('blood_pressure', TEXT(), table=<health>), Column('stress_level', BIGINT(), table=<health>), Column('heart_rate', BIGINT(), table=<health>), Column('systolic', BIGINT(), 

In [14]:
for mappedclass in Base.classes:
    print(mappedclass)



In [15]:
for mdtable in Base.metadata.tables:
    print(mdtable)

activity
bmi
gender
health
occupation
person
sleep
sleep_disorder


In [18]:
# session= Session(engine)
Person= pd.read_sql("SELECT * FROM person", conn)
Sleep= pd.read_sql("SELECT * FROM sleep", conn)
Health= pd.read_sql("SELECT * FROM health", conn)
merged_data = pd.merge(Person, Sleep, on='id')
merged_data = pd.merge(merged_data, Health, on='id')
merged_data


,index_x,id,gender_id,age,occupation_id,index_y,person_id_x,sleep_duration,sleep_quality,sleep_disorder_id,index,person_id_y,blood_pressure,stress_level,heart_rate,systolic,diastolic,bmi_id
0,0,1,0,27,0,1,2,6.2,6,0,1,69,128/85,6,76,128,85,0
1,1,6,0,28,0,6,11,6.1,6,0,6,84,128/84,5,70,128,84,0
2,2,85,0,35,0,85,101,7.2,8,0,85,275,140/95,8,75,140,95,0
3,3,93,0,35,0,93,112,7.4,8,0,93,286,140/95,8,75,140,95,0
4,4,2,0,28,1,2,3,6.2,6,0,2,70,128/85,6,76,128,85,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368,369,69,1,33,7,69,83,6.7,7,0,69,257,135/90,4,65,135,90,0
369,370,70,1,33,7,70,84,6.7,7,0,70,258,135/90,4,65,135,90,0
370,371,81,1,34,7,81,97,7.2,8,0,81,271,140/95,8,75,140,95,0
371,372,82,1,34,7,82,98,7.1,8,0,82,272,140/95,8,75,140,95,0


In [21]:
Health

,index,id,person_id,blood_pressure,stress_level,heart_rate,systolic,diastolic,bmi_id
0,0,0,1,126/83,6,77,126,83,0
1,1,1,69,128/85,6,76,128,85,0
2,2,2,70,128/85,6,76,128,85,0
3,3,3,81,131/86,8,81,131,86,0
4,4,4,82,131/86,8,81,131,86,0
...,...,...,...,...,...,...,...,...,...
369,369,369,146,135/88,5,84,135,88,2
370,370,370,265,142/92,5,83,142,92,2
371,371,371,267,142/92,5,83,142,92,2
372,372,372,277,139/91,3,86,139,91,2


In [24]:
meetrics = ['blood_pressure','heart_rate','systolic','diastolic']
Health[meetrics]

,blood_pressure,heart_rate,systolic,diastolic
0,126/83,77,126,83
1,128/85,76,128,85
2,128/85,76,128,85
3,131/86,81,131,86
4,131/86,81,131,86
...,...,...,...,...
369,135/88,84,135,88
370,142/92,83,142,92
371,142/92,83,142,92
372,139/91,86,139,91


In [22]:
health_metrics = Health['blood_pressure','heart_rate','systolic','diastolic']
health_metrics


IndexingError: Too many indexers